# Analisi sulle prestazioni

In questo notebook si vuole svolgere un'analisi sulle performance del modulo
implementato. Andremo quindi a confrontare:

- Versione sequenziale e versione parallela.
- Versione sequenziale e DEAP sequenziale.
- Versione parallela e DEAP parallelo (`scoop` e `multiprocessing`).
- Considerazioni finali sul problema specifico del paper per un corretto
  utilizzo della versione parallela ed evitare errori.
